In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import yfinance as yf
import pandas as pd
import polars as pl
from typing import Dict
from dataclasses import asdict, is_dataclass, dataclass
import scipy
import numpy as np
from scipy.stats import linregress

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader
from src.common.AssetFileInOut import AssetFileInOut
from src.common.AssetDataPolars import AssetDataPolars


In [9]:
import os
print(os.getcwd())

c:\Users\kimer\Desktop\RandomOdyssey\notebooks


In [ ]:
#load api key
import yaml
import os

# Define paths
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(parent_dir, desired_folder)

# Path to the YAML file
yaml_file_path = os.path.join(absolute_path_to_folder, "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_acc']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)
print(apiKey)

API Key: your_api_key_here
your_api_key_here
